In [1]:
import pandas as pd

1. Transform state_data.xlsx

In [46]:
G = ['Male '*5,'Female '*5,'Person '*5]
Gender = []
for gender in G:
    l = gender.split(' ')
    for g in l:
        if len(g) != 0 :
            Gender.append(g)
Gender = Gender*3
job = ['Number of Jobs'] * 15
income = ['Median Income per Job'] * 15
adjusted_income = ['Duration Adjusted Income per Job']*15
Category= []
Category.extend(job)
Category.extend(income)
Category.extend(adjusted_income)


In [47]:
newData = pd.read_excel('state_data.xlsx', sheet_name='Sheet1')
newData = newData.drop(['Unnamed: 1'], axis=1)
df_transposed = newData.T
df_transposed.columns = df_transposed.iloc[0]
df_transposed = df_transposed[1:]
col = df_transposed.columns.tolist()
col[0] = 'Year'
df_transposed.columns = col
df_transposed['Gender'] = Gender
df_transposed['Category'] = Category
df_transposed.reset_index(drop=True)
df_transposed.index = range(len(df_transposed.index))

In [59]:
df_transposed.to_excel('output.xlsx')

In [69]:
states = df_transposed.columns[2:10].tolist()
state_abbreviations_list  = ['NSW', 'VIC', 'QLD', 'SA', 'WA', 'TAS', 'NT', 'ACT']

In [70]:
dfs = []

for i in range(len(states)):
    df = df_transposed[[states[i],'Year','Category','Gender']]
    df.columns = ['Value','Year','Category','Gender']
    df['State'] = state_abbreviations_list[i]
    dfs.append(df)


<ipython-input-70-ac72df8564cf>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['State'] = state_abbreviations_list[i]


In [86]:
df = pd.concat(dfs)
df.to_excel('output.xlsx',index=False)

3. Transform state2.xlsx

In [90]:
newData = pd.read_excel('state2.xlsx')
newData = newData.drop(['STATE NAME'], axis=1)
newData.columns[-1]

'2019-20'

In [91]:
new_df = newData.pivot_table(index=['STATE', 'Age Range'], columns='Sex', values='2019-20').reset_index()

In [94]:
new_df.to_excel('2020_median_income.xlsx')
new_df.head()

Sex,STATE,Age Range,Females,Males,Persons
0,ACT,24 and Under,26465,29531,27863
1,ACT,25 to 34,64913,71465,68212
2,ACT,35 to 44,79387,100005,89617
3,ACT,45 to 54,87520,109828,97089
4,ACT,55 to 64,74100,97990,84993


3. Transform Employee jobs and employee income.xlsx

In [ ]:
# Read the 'Sheet1' sheet from the Excel file
df = pd.read_excel('Employee jobs and employee income.xlsx', sheet_name='Table 2.5')
df_selected = df[4:2304]
df_selected = df_selected.reset_index(drop=True)
NSW_row_num = df_selected.index[df_selected[df_selected.columns[0]] == 'New South Wales'][0]
VIC_row_num = df_selected.index[df_selected[df_selected.columns[0]] == 'Victoria'][0]
QLD_row_num = df_selected.index[df_selected[df_selected.columns[0]] == 'Queensland'][0]
SA_row_num = df_selected.index[df_selected[df_selected.columns[0]] == 'South Australia'][0]
WA_num = df_selected.index[df_selected[df_selected.columns[0]] == 'Western Australia'][0]
TAS_row_num = df_selected.index[df_selected[df_selected.columns[0]] == 'Tasmania'][0]
NT_row_num = df_selected.index[df_selected[df_selected.columns[0]] == 'Northern Territory'][0]
ACT_row_num = df_selected.index[df_selected[df_selected.columns[0]] == 'Australian Capital Territory'][0]



In [ ]:
def custom_conversion(value):
    # Remove commas from the string
    value = str(value).replace(',', '')
    
    # Check if the string is a numeric value
    if value.isdigit():
        return int(value)
    else:
        return 0

In [ ]:
NSW_data = df_selected[NSW_row_num:VIC_row_num]

VIC_data = df_selected[VIC_row_num:QLD_row_num]

QLD_data = df_selected[QLD_row_num:SA_row_num]

SA_data = df_selected[SA_row_num:WA_num]

WA_data = df_selected[WA_num:TAS_row_num]

TAS_data = df_selected[TAS_row_num:NT_row_num]

NT_DATA = df_selected[NT_row_num:ACT_row_num]

ACT_DATA = df_selected[ACT_row_num:]

df_list = [NSW_data, VIC_data,QLD_data, SA_data, WA_data, TAS_data, NT_DATA, ACT_DATA]

for i in range(len(df_list)):
    df_list[i] = df_list[i].reset_index(drop=True)
    df_list[i].loc[0, 'Unnamed: 1'] = df_list[i].loc[0, df_list[i].columns[0]]
    df_list[i] = df_list[i].drop(df_list[i].columns[0], axis=1)
    df_list[i] = df_list[i].rename(columns={'Unnamed: 1': 'Location'})
    df_list[i]['Unnamed: 21'] = df_list[i]['Unnamed: 21'].apply(custom_conversion)
    df_list[i]['Unnamed: 26'] = df_list[i]['Unnamed: 26'].apply(custom_conversion)

In [ ]:
NSW_data = df_list[0][['Location', 'Unnamed: 21', 'Unnamed: 26']]
NSW_data['STATE'] = 'NSW'
NSW_data.rename(columns={'Unnamed: 21': 'male_median_salary', 'Unnamed: 26': 'female_median_salary'}, inplace=True)
NSW_data = NSW_data.astype({'male_median_salary': int, 'female_median_salary': int})
NSW_data['gender_pay_gap'] = ((NSW_data['male_median_salary'] - NSW_data['female_median_salary']) / NSW_data['male_median_salary']) * 100


VIC_data = df_list[1][['Location', 'Unnamed: 21', 'Unnamed: 26']]
VIC_data['STATE'] = 'VIC'
VIC_data.rename(columns={'Unnamed: 21': 'male_median_salary', 'Unnamed: 26': 'female_median_salary'}, inplace=True)
VIC_data = VIC_data.astype({'male_median_salary': int, 'female_median_salary': int})
VIC_data['gender_pay_gap'] = ((VIC_data['male_median_salary'] - VIC_data['female_median_salary']) / VIC_data['male_median_salary']) * 100

QLD_data = df_list[2][['Location', 'Unnamed: 21', 'Unnamed: 26']]
QLD_data['STATE'] = 'QLD'
QLD_data.rename(columns={'Unnamed: 21': 'male_median_salary', 'Unnamed: 26': 'female_median_salary'}, inplace=True)
QLD_data = QLD_data.astype({'male_median_salary': int, 'female_median_salary': int})
QLD_data['gender_pay_gap'] = ((QLD_data['male_median_salary'] - QLD_data['female_median_salary']) / QLD_data['male_median_salary']) * 100

SA_data = df_list[3][['Location', 'Unnamed: 21', 'Unnamed: 26']]
SA_data['STATE'] = 'SA'
SA_data.rename(columns={'Unnamed: 21': 'male_median_salary', 'Unnamed: 26': 'female_median_salary'}, inplace=True)
SA_data = SA_data.astype({'male_median_salary': int, 'female_median_salary': int})
SA_data['gender_pay_gap'] = ((SA_data['male_median_salary'] - SA_data['female_median_salary']) / SA_data['male_median_salary']) * 100

WA_data = df_list[4][['Location', 'Unnamed: 21', 'Unnamed: 26']]
WA_data['STATE'] = 'WA'
WA_data.rename(columns={'Unnamed: 21': 'male_median_salary', 'Unnamed: 26': 'female_median_salary'}, inplace=True)
WA_data = WA_data.astype({'male_median_salary': int, 'female_median_salary': int})
WA_data['gender_pay_gap'] = ((WA_data['male_median_salary'] - WA_data['female_median_salary']) / WA_data['male_median_salary']) * 100

TAS_data = df_list[5][['Location', 'Unnamed: 21', 'Unnamed: 26']]
TAS_data['STATE'] = 'TAS'
TAS_data.rename(columns={'Unnamed: 21': 'male_median_salary', 'Unnamed: 26': 'female_median_salary'}, inplace=True)
TAS_data = TAS_data.astype({'male_median_salary': int, 'female_median_salary': int})
TAS_data['gender_pay_gap'] = ((TAS_data['male_median_salary'] - TAS_data['female_median_salary']) / TAS_data['male_median_salary']) * 100

NT_DATA = df_list[6][['Location', 'Unnamed: 21', 'Unnamed: 26']]
NT_DATA['STATE'] = 'NT'
NT_DATA.rename(columns={'Unnamed: 21': 'male_median_salary', 'Unnamed: 26': 'female_median_salary'}, inplace=True)
NT_DATA = NT_DATA.astype({'male_median_salary': int, 'female_median_salary': int})
NT_DATA['gender_pay_gap'] = ((NT_DATA['male_median_salary'] - NT_DATA['female_median_salary']) / NT_DATA['male_median_salary']) * 100

ACT_DATA = df_list[7][['Location', 'Unnamed: 21', 'Unnamed: 26']]
ACT_DATA['STATE'] = 'ACT'
ACT_DATA.rename(columns={'Unnamed: 21': 'male_median_salary', 'Unnamed: 26': 'female_median_salary'}, inplace=True)
ACT_DATA = ACT_DATA.astype({'male_median_salary': int, 'female_median_salary': int})
ACT_DATA['gender_pay_gap'] = ((ACT_DATA['male_median_salary'] - ACT_DATA['female_median_salary']) / ACT_DATA['male_median_salary']) * 100

In [ ]:
def replace_negative_and_inf(value):
    if value < 0 or np.isinf(value) or  value == 100:
        return 0
    else:
        return value

In [ ]:
df_list = [NSW_data, VIC_data,QLD_data, SA_data, WA_data, TAS_data, NT_DATA, ACT_DATA]
df = pd.concat(df_list, axis =0)
df['gender_pay_gap'] = df['gender_pay_gap'].apply(replace_negative_and_inf)
df = df.reset_index(drop=True)
df = df[df['gender_pay_gap'] != 0]
df = df.dropna(subset=['gender_pay_gap'])
df = df.reset_index(drop=True)
df = df.rename(columns={'STATE': 'state', 'Location':'location'})
list_of_dicts = df.to_dict(orient='records')

In [ ]:
import json
def format_dict_without_quotes(d):
    formatted_dict = '{' + ', '.join(f'{key}: {json.dumps(value)}' for key, value in d.items()) + '}'
    return formatted_dict

In [ ]:

# Serialize the list of dictionaries as a JSON-formatted string
list_of_dicts_json = json.dumps(list_of_dicts)

# Write the JSON-formatted string to a text file
with open('java_data.txt', 'w') as file:
    file.write('[\n')
    for record in list_of_dicts:
        file.write('    ')
        file.write(format_dict_without_quotes(record) + ', \n')
    file.write('];')

4. Full-time non-managerial employees paid at the adult rate, average weekly total cash earnings - industry by sex.xlsx

In [6]:
df = pd.read_excel('Full-time non-managerial employees paid at the adult rate, average weekly total cash earnings - industry by sex.xlsx')

In [7]:
df_female = df[['INDUSTRY', 'Females ($)']].rename(columns={'Females ($)': 'salary'})
df_female['Gender'] = 'F'
df_male = df[['INDUSTRY', 'Males ($)']].rename(columns={'Males ($)': 'salary'})
df_male['Gender'] = 'M'
df = pd.concat([df_female,df_male], axis =0)
df.to_csv('industry.csv',index=False)

In [8]:
df.to_csv('industry.csv',index=False)

In [3]:
6/13

0.46153846153846156

In [15]:
import numpy as np
a = np.array([12 , 10, 13, 8,10,12,5])
b= np.array([25,12,26,26,12,19,17])

In [16]:
a/b

array([0.48      , 0.83333333, 0.5       , 0.30769231, 0.83333333,
       0.63157895, 0.29411765])

In [13]:
7260.0 * 0.2 /3

484.0

In [17]:
sum(a)

70

In [23]:
183.75 * ((12 + 2.5)/70)/0.35

108.75

In [28]:
((12 + 0.75)/70)/0.35

0.5204081632653061

In [33]:
183.75 * (1 - 0.5204081632653061)

88.125

In [27]:
10 * 0.3 * 0.25

0.75